In [ ]:
library(Seurat)
library(SeuratDisk)
library(patchwork)
library(harmony)
library(reshape2)
library(RColorBrewer)
library(dplyr)
library(readr)
library(tidyverse)
library(Matrix)
library(Seurat)
library(ggplot2)
library(cowplot)
library(celldex)
library(SingleR)
library(data.table)
library(scGate)
library(ggpubr)
library(scCustomize)
library(XeniumSpatialAnalysis)

# Loading data

In [ ]:
load("Xenium/NSCLC.1B.rda")
load("Xenium/nsclc_2_a.rda")
load("Xenium/hgsoc_1_b.rda")
load("Xenium/hgsoc_2_a.rda")


In [ ]:
ls()

In [ ]:
nsclc_1B  <- NSCLC.1B

In [ ]:
rm(NSCLC.1B)

In [ ]:
ls()

## Markers in the panel

In [ ]:
rownames(hgsoc_1_b)

In [ ]:
ImageDimPlot(hgsoc_1_b, fov = "fov", molecules = c("LCK", "CD3E", "FOXP3", "CD79A"), nmols = 20000, axes = )

In [ ]:
hgsoc_2_a@images$fov

In [ ]:
str(hgsoc_2_a@images$fov)

In [ ]:
ImageDimPlot(hgsoc_2_a, fov = "fov", molecules = c("LCK", "CD3E", "FOXP3", "CD79A"), nmols = 20000)

In [ ]:
ImageDimPlot(nsclc_1B, fov = "fov", molecules = c("LCK", "CD3E", "FOXP3", "CD79A"), nmols = 20000)

In [ ]:
ImageDimPlot(nsclc_2_a, fov = "fov", molecules = c("LCK", "CD3E", "FOXP3", "CD79A"), nmols = 20000)

# Clustering, marker visualization

In [ ]:
hgsoc_1_b <- SCTransform(hgsoc_1_b, assay = "Xenium")
hgsoc_1_b <- RunPCA(hgsoc_1_b, npcs = 30, features = rownames(hgsoc_1_b))
hgsoc_1_b <- RunUMAP(hgsoc_1_b, dims = 1:30)
hgsoc_1_b <- FindNeighbors(hgsoc_1_b, reduction = "pca", dims = 1:30)
hgsoc_1_b <- FindClusters(hgsoc_1_b, resolution = 0.3)

In [ ]:
DimPlot(hgsoc_1_b)

In [ ]:
FeaturePlot(hgsoc_1_b, features = c("CD8A","CD4","LCK","CD3D"))

In [ ]:
FeaturePlot(hgsoc_1_b, features = c("CD79A","MS4A1","JCHAIN","FCER2"))

In [ ]:
dir.create("./figures/dimplots/hgsoc_1_b/", recursive = T)

In [ ]:
for(i in rownames(hgsoc_1_b)){
    print(FeaturePlot(hgsoc_1_b, features = i))
    ggsave(paste0("figures/dimplots/hgsoc_1_b/",i,".png"), width = 13, height = 10, units = "cm")
}

In [ ]:
hgsoc_2_a <- SCTransform(hgsoc_2_a, assay = "Xenium")
hgsoc_2_a <- RunPCA(hgsoc_2_a, npcs = 30, features = rownames(hgsoc_2_a))
hgsoc_2_a <- RunUMAP(hgsoc_2_a, dims = 1:30)
hgsoc_2_a <- FindNeighbors(hgsoc_2_a, reduction = "pca", dims = 1:30)
hgsoc_2_a <- FindClusters(hgsoc_2_a, resolution = 0.3)

In [ ]:
dir.create("./figures/dimplots/hgsoc_2_a/", recursive = T)

In [ ]:
for(i in rownames(hgsoc_2_a)){
    print(FeaturePlot(hgsoc_2_a, features = i))
    ggsave(paste0("figures/dimplots/hgsoc_2_a/",i,".png"), width = 13, height = 10, units = "cm")
}

In [ ]:
nsclc_1B <- SCTransform(nsclc_1B, assay = "Xenium")
nsclc_1B <- RunPCA(nsclc_1B, npcs = 30, features = rownames(nsclc_1B))
nsclc_1B <- RunUMAP(nsclc_1B, dims = 1:30)
nsclc_1B <- FindNeighbors(nsclc_1B, reduction = "pca", dims = 1:30)
nsclc_1B <- FindClusters(nsclc_1B, resolution = 0.3)

In [ ]:
dir.create("./figures/dimplots/nsclc_1B/", recursive = T)

In [ ]:
for(i in rownames(nsclc_1B)){
    print(FeaturePlot(nsclc_1B, features = i))
    ggsave(paste0("figures/dimplots/nsclc_1B/",i,".png"), width = 13, height = 10, units = "cm")
}

In [ ]:
options(future.globals.maxSize = 20000 * 1024^2)

In [ ]:
nsclc_2_a <- SCTransform(nsclc_2_a, assay = "Xenium")
nsclc_2_a <- RunPCA(nsclc_2_a, npcs = 30, features = rownames(nsclc_2_a))
nsclc_2_a <- RunUMAP(nsclc_2_a, dims = 1:30)
nsclc_2_a <- FindNeighbors(nsclc_2_a, reduction = "pca", dims = 1:30)
nsclc_2_a <- FindClusters(nsclc_2_a, resolution = 0.3)

In [ ]:
nsclc_2_a

In [ ]:
dir.create("./figures/dimplots/nsclc_2_a/", recursive = T)

In [ ]:
for(i in rownames(nsclc_2_a)){
    print(FeaturePlot(nsclc_2_a, features = i))
    ggsave(paste0("figures/dimplots/nsclc_2_a/",i,".png"), width = 13, height = 10, units = "cm")
}

In [ ]:
dir.create("./datasets/", recursive = T)

In [ ]:
saveRDS(nsclc_2_a, "./datasets/nsclc_2_a.rds")
saveRDS(nsclc_1B, "./datasets/nsclc_1B.rds")
saveRDS(hgsoc_2_a, "./datasets/hgsoc_2_a.rds")
saveRDS(hgsoc_1_b, "./datasets/hgsoc_1_b.rds")

# Identification of cell types

## Downsampling for easier optimization

In [ ]:
nsclc_2_a$barcode  <- colnames(nsclc_2_a)
nsclc_1B$barcode  <- colnames(nsclc_1B)
hgsoc_2_a$barcode  <- colnames(hgsoc_2_a)
hgsoc_1_b$barcode  <- colnames(hgsoc_1_b)


In [ ]:
nsclc_2_a_ds  <- subset(nsclc_2_a, barcode %in% sample(colnames(nsclc_2_a), size = 20000))

In [ ]:
nsclc_1B_ds  <- subset(nsclc_1B, barcode %in% sample(colnames(nsclc_1B), size = 20000))

In [ ]:
hgsoc_2_a_ds  <- subset(hgsoc_2_a, barcode %in% sample(colnames(hgsoc_2_a), size = 20000))

In [ ]:
hgsoc_1_b_ds  <- subset(hgsoc_1_b, barcode %in% sample(colnames(hgsoc_1_b), size = 20000))

## Merging samples based on cancer type

In [ ]:
nsclc_2_a_ds$sample  <- "nsclc_2_a"
nsclc_1B_ds$sample  <- "nsclc_1B"
hgsoc_1_b_ds$sample  <- "hgsoc_1_b"
hgsoc_2_a_ds$sample  <- "hgsoc_2_a"


### Lung

In [ ]:
int_lung  <- merge(nsclc_2_a_ds, nsclc_1B_ds)

In [ ]:
int_lung <- SCTransform(int_lung, assay = "Xenium")
int_lung <- RunPCA(int_lung, npcs = 30, features = rownames(int_lung))
int_lung <- RunUMAP(int_lung, dims = 1:30)
int_lung <- FindNeighbors(int_lung, reduction = "pca", dims = 1:30)
int_lung <- FindClusters(int_lung, resolution = 0.3)

In [ ]:
DimPlot(int_lung, group.by = "sample")

In [ ]:
int_lung <- split(int_lung, f = int_lung$sample)

In [ ]:
int_lung <- NormalizeData(int_lung)
int_lung <- FindVariableFeatures(int_lung)
int_lung <- ScaleData(int_lung)
int_lung <- RunPCA(int_lung)

In [ ]:
int_lung <- IntegrateLayers(
  object = int_lung, method = HarmonyIntegration,
  orig.reduction = "pca", new.reduction = "harmony",
  verbose = FALSE
)

In [ ]:
int_lung <- RunUMAP(int_lung, reduction = "harmony", dims = 1:30, reduction.name = "umap.harmony")

In [ ]:
DimPlot(int_lung, group.by = "sample", reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_lung, features = c("CD79A","MS4A1","JCHAIN","FCER2"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_lung, features = c("CD8A","CD4","LCK","CD3D"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_lung, features = c("CEACAM1","EGFR","EPCAM","KRAS"), reduction = "umap.harmony")

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
FeaturePlot(int_lung, features = c('KLRB1','KLRC1','KLRD1','KLRF1','KLRK1',
                                  "NCAM1","NKG7","EOMES",
                                  'GNLY','GZMA','GZMB','GZMH','GZMK'), reduction = "umap.harmony", ncol = 7)

### Ovarian

In [ ]:
int_ova  <- merge(hgsoc_1_b_ds, hgsoc_2_a_ds)

In [ ]:
int_ova <- SCTransform(int_ova, assay = "Xenium")
int_ova <- RunPCA(int_ova, npcs = 30, features = rownames(int_ova))
int_ova <- RunUMAP(int_ova, dims = 1:30)
int_ova <- FindNeighbors(int_ova, reduction = "pca", dims = 1:30)
int_ova <- FindClusters(int_ova, resolution = 0.3)

In [ ]:
DimPlot(int_ova, group.by = "sample")

In [ ]:
int_ova <- split(int_ova, f = int_ova$sample)

In [ ]:
int_ova <- NormalizeData(int_ova)
int_ova <- FindVariableFeatures(int_ova)
int_ova <- ScaleData(int_ova)
int_ova <- RunPCA(int_ova)

In [ ]:
int_ova <- IntegrateLayers(
  object = int_ova, method = HarmonyIntegration,
  orig.reduction = "pca", new.reduction = "harmony",
  verbose = FALSE
)

In [ ]:
int_ova <- RunUMAP(int_ova, reduction = "harmony", dims = 1:30, reduction.name = "umap.harmony")

In [ ]:
DimPlot(int_ova, group.by = "sample", reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_ova, features = c("CD79A","MS4A1","JCHAIN","FCER2"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_ova, features = c("CD8A","CD4","LCK","CD3D"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_ova, features = c("CEACAM1","EGFR","EPCAM","KRAS"), reduction = "umap.harmony")

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
FeaturePlot(int_ova, features = c('KLRB1','KLRC1','KLRD1','KLRF1','KLRK1',
                                  "NCAM1","NKG7","EOMES",
                                  'GNLY','GZMA','GZMB','GZMH','GZMK'), reduction = "umap.harmony", ncol = 7)

In [ ]:
rm(int_ova, hgsoc_1_b, hgsoc_1_b_ds, hgsoc_2_a, hgsoc_2_a_ds, nsclc_1B_ds, nsclc_2_a_ds)

In [ ]:
gc()

# Lung cancer 

## Full object integration

In [ ]:
int_lung  <- merge(nsclc_2_a, nsclc_1B)

In [ ]:
int_lung <- SCTransform(int_lung, assay = "Xenium")
int_lung <- RunPCA(int_lung, npcs = 30, features = rownames(int_lung))
int_lung <- RunUMAP(int_lung, dims = 1:30)
int_lung <- FindNeighbors(int_lung, reduction = "pca", dims = 1:30)
int_lung <- FindClusters(int_lung, resolution = 0.3)

In [ ]:
head(colnames(int_lung))

In [ ]:
tail(colnames(int_lung))

In [ ]:
int_lung$sample  <- substr(colnames(int_lung),12,12)

In [ ]:
nsclc_2_a
nsclc_1B

In [ ]:
int_lung$sample  <- ifelse(int_lung$sample == 1, "nsclc_2_a","nsclc_1B")

In [ ]:
int_lung$sample  %>% table

In [ ]:
DimPlot(int_lung, group.by = "seurat_clusters", label = TRUE, label.size = 8)

In [ ]:
DimPlot(int_lung, group.by = "sample", label = TRUE, label.size = 8)

In [ ]:
VlnPlot(int_lung, features =  c("nFeature_Xenium", "nCount_Xenium"), pt.size = 0)

In [ ]:
int_lung$keep  <- ifelse(int_lung$seurat_clusters %in% c(19,22),"Remove","Keep")

In [ ]:
DimPlot(int_lung, group.by = "keep", cols = c("grey88",'red'))

In [ ]:
ggplot(data.frame(nCount_Xenium = int_lung$nCount_Xenium,
                  nFeature_Xenium = int_lung$nFeature_Xenium,
                  seurat_clusters = int_lung$seurat_clusters,
                        exclude = ifelse(int_lung$keep == "Keep", "Keep", "Remove")), 
       aes(x = seurat_clusters, y = nFeature_Xenium)) +
  geom_violin(scale = "width", aes(fill = exclude)) + 
  ggtitle("Excluded clusters of low quality cells") + 
  theme_classic() +
  scale_fill_manual(values = c("white","red")) +
  theme(panel.background = element_blank(), 
        axis.text.x = element_text(angle = 0, hjust = 1))

In [ ]:
int_lung$keep  %>% table

In [ ]:
table(int_lung$keep, int_lung$sample)

In [ ]:
saveRDS(int_lung, "./datasets/lung_integrated_full.rds")

In [ ]:
int_lung  <- subset(int_lung, keep == "Keep")

In [ ]:
DimPlot(int_lung)

In [ ]:
DimPlot(int_lung, group.by = "sample")

In [ ]:
rm(nsclc_1B)
rm(nsclc_2_a)


In [ ]:
gc()

In [ ]:
int_lung <- split(int_lung, f = int_lung$sample)

In [ ]:
int_lung <- NormalizeData(int_lung)
int_lung <- FindVariableFeatures(int_lung)
int_lung <- ScaleData(int_lung)
int_lung <- RunPCA(int_lung)

In [ ]:
int_lung

In [ ]:
int_lung <- IntegrateLayers(
  object = int_lung, method = HarmonyIntegration,
  orig.reduction = "pca", new.reduction = "harmony",
  verbose = FALSE
)

In [ ]:
int_lung <- RunUMAP(int_lung, reduction = "harmony", dims = 1:30, reduction.name = "umap.harmony")

In [ ]:
FeaturePlot(int_lung, features = c("CD79A","MS4A1","JCHAIN","FCER2"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_lung, features = c("CD8A","CD4","LCK","CD3D"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_lung, features = c("CEACAM1","EGFR","EPCAM","KRAS"), reduction = "umap.harmony")

In [ ]:
options(repr.plot.width = 24, repr.plot.height = 8)
FeaturePlot(int_lung, features = c('KLRB1','KLRC1','KLRD1','KLRF1','KLRK1',
                                  "NCAM1","NKG7","EOMES",
                                  'GNLY','GZMA','GZMB','GZMH','GZMK'), reduction = "umap.harmony", ncol = 7)

In [ ]:
saveRDS(int_lung, "./datasets/lung_integrated_full_filt_harmony.rds")

In [ ]:
rm(int_lung)
gc()

In [ ]:
int_lung  <- readRDS("./datasets/lung_integrated_full_filt_harmony.rds")

In [ ]:
DimPlot(int_lung, reduction = "umap.harmony", label  = T, label.size = 8)

In [ ]:
VlnPlot(int_lung, features =  c("nFeature_Xenium", "nCount_Xenium"), pt.size = 0)

In [ ]:
VlnPlot(int_lung, features =  c("CD3D", "LCK"), pt.size = 0)

In [ ]:
int_lung <- RunUMAP(int_lung, reduction = "harmony", dims = 1:30, reduction.name = "umap.harmony")

In [ ]:
DimPlot(int_lung, reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_lung, features = c("CD79A","MS4A1","JCHAIN","FCER2"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_lung, features = c("CD8A","CD4","LCK","CD3D"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_lung, features = c("CEACAM1","EGFR","EPCAM","KRAS"), reduction = "umap.harmony")

In [ ]:
options(repr.plot.width = 40, repr.plot.height = 30)
FeaturePlot(int_lung, features = c('KLRB1','KLRC1','KLRD1','KLRF1','KLRK1',
                                  "NCAM1","NKG7","EOMES",
                                  'GNLY','GZMA','GZMB','GZMH','GZMK', "CD3D","NCR1"), reduction = "umap.harmony", ncol = 4)

In [ ]:
int_lung  <- FindNeighbors(int_lung)

In [ ]:
int_lung  <- FindClusters(int_lung, resolution = 0.2)

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 9)

DimPlot(int_lung, reduction = "umap.harmony", label = T, label.size = 8)

In [ ]:
FeaturePlot(int_lung, features = c("KLRF1","GNLY","CD3D","EPCAM"), reduction = "umap.harmony")

## Annotation of cell types

In [ ]:
mid.se <- celldex::MonacoImmuneData()
hpca.se  <- celldex::HumanPrimaryCellAtlasData()



In [ ]:
int_lung

In [ ]:
int_lung  <- JoinLayers(int_lung)

In [ ]:
 DefaultAssay(int_lung)  <- "SCT"
	
    ### Annotate the dataset with Monaco Immune dataset
counts  <- int_lung@assays$SCT@layers$counts

In [ ]:
rownames(counts)  <- rownames(int_lung@assays$SCT)

In [ ]:
		pred.singler <- SingleR(test = counts, ref = mid.se, assay.type.test=1,
		labels = mid.se$label.fine, fine.tune = F)
    

    	### Annotate the dataset with Wherry dataset
        pred.singler3 <- SingleR(test =  counts, ref = hpca.se,
		assay.type.test=1,
		labels = hpca.se$label.fine, fine.tune = F)

		all_labels <- data.frame(
		Monaco_single = pred.singler$labels,
		HPCA_single = pred.singler3$labels,
		
		barcode = colnames(int_lung))
		
		md2 <- int_lung@meta.data
		md2$barcode = colnames(int_lung)

		md3 <- left_join(md2, all_labels)
rownames(md3) <- colnames(int_lung)
		int_lung@meta.data <- md3
		

In [ ]:
prediction_monaco  <- pred.singler$scores  %>% as.data.frame()

In [ ]:
prediction_monaco$cluster  <- unname(int_lung$seurat_clusters)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 7.5)
DimPlot(int_lung, raster = F, group.by = "Monaco_single", label = F, repel = T, reduction = "umap.harmony")

In [ ]:
DimPlot(int_lung, raster = F, 
        cells.highlight = colnames(subset(int_lung, Monaco_single == "Natural killer cells")),
        reduction = "umap.harmony")

In [ ]:
prediction_monaco  <- prediction_monaco  %>% pivot_longer(!cluster, 
                                                          names_to = "annotation", values_to = "score")

In [ ]:
options(repr.plot.width = 17, repr.plot.height = 10)

ggplot(prediction_monaco, aes(x = cluster, y = score)) +
  geom_violin(scale = "width", aes(fill = cluster)) +
  facet_wrap(~annotation, ncol = 8) +
  ggtitle("Cluster annotations - Monaco Immune Database") + 
  theme_classic() +
  theme(panel.background = element_blank(), 
        axis.text.x = element_text(angle = 0, hjust = 1))

In [ ]:
options(repr.plot.width = 40, repr.plot.height = 10)
DimPlot(int_lung, raster = F, group.by = "HPCA_single", label = F, reduction = "umap.harmony")

In [ ]:
int_lung$HPCA_single  %>% table

In [ ]:
int_lung$hpca_nk_cell  <- ifelse(grepl(int_lung$HPCA_single, pattern = "NK_cell"), 
       int_lung$HPCA_single, "Other") 

In [ ]:
int_lung$hpca_nk_cell  %>% table

In [ ]:
DimPlot(int_lung, group.by = "hpca_nk_cell", reduction = "umap.harmony", cols = c("red","blue","green","grey88"))

In [ ]:
DimPlot(int_lung, raster = F, 
        cells.highlight = colnames(subset(int_lung, hpca_nk_cell == "NK_cell")),
        reduction = "umap.harmony")

In [ ]:
DimPlot(int_lung, raster = F, 
        cells.highlight = colnames(subset(int_lung, hpca_nk_cell == "NK_cell:IL2")),
        reduction = "umap.harmony")

In [ ]:
saveRDS(int_lung, "./datasets//lung_integrated_full_filt_harmony.rds")

In [ ]:
reduced_annot  <- function(seurat_object, tier_name, n_cells_to_other){
    md  <- seurat_object@meta.data
    tier_name_red  <- paste0(tier_name,"_red")
    cell_types_keep  <- (md  %>% group_by_at(tier_name)  %>% tally()  %>% arrange(desc(n)) %>% dplyr::filter(n > n_cells_to_other))[[tier_name]]
    md[[tier_name_red]]  <- if_else(md[[tier_name]] %in% cell_types_keep,as.character(md[[tier_name]]),"Other")
    seurat_object  <- AddMetaData(seurat_object, metadata = md[[tier_name_red]], tier_name_red)
    return(seurat_object)
}


In [ ]:
int_lung$hpca_main  <-  sub("\\:.*", "", int_lung$HPCA_single)

In [ ]:
int_lung$hpca_main  %>% table

In [ ]:
options(repr.plot.width = 19, repr.plot.height = 9)
DimPlot(int_lung, group.by = "hpca_main", reduction = "umap.harmony", label = T)

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 9)

DimPlot(int_lung, reduction = "umap.harmony", label = T, label.size = 8)

In [ ]:
FeaturePlot(int_lung,  reduction = "umap.harmony",
           features = c("CD3D","CD19","DCN","FN1","CD68","AIF1","MARCO","VSIG4"), ncol = 4)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 4)
DotPlot(int_lung, cols = c("dodgerblue","indianred2"),
        features = c(
            "ARID1A","EPCAM","CEACAM8",
            "CD3D","CD3E","CD8A","CD4",
            "CD19","BANK1","MS4A1","CD79A",
            "CD27","CD38",
            "FN1",
            "CD68","AIF1","MARCO","VSIG4",
            "CD1A","CD1C","CD80","CD86"
        )) + theme(axis.text.x = element_text(angle = 90))

In [ ]:
mrk  <- FindAllMarkers(int_lung, only.pos = TRUE, logfc.threshold = log(2))

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6.5)
dir.create("./figures/dimplots/int_lung/", recursive = T)

for(i in rownames(int_lung)){
    print(FeaturePlot(int_lung, features = i, min.cutoff = 0, reduction = "umap.harmony"))
    ggsave(paste0("figures/dimplots/int_lung/",i,".png"), width = 13, height = 10, units = "cm")
}

## Coarse annotations

In [ ]:
int_lung  <- readRDS("./datasets/lung_integrated_full_filt_harmony.rds")

In [ ]:
int_lung$annot

## T NK reclustering

In [ ]:
t_nk_lung  <- subset(int_lung, seurat_clusters == 3)

In [ ]:
t_nk_lung <- RunUMAP(t_nk_lung, reduction = "harmony", dims = 1:30, reduction.name = "umap.harmony")

In [ ]:
DimPlot(t_nk_lung, group.by = "sample", reduction = "umap.harmony")

In [ ]:
FeaturePlot(t_nk_lung, features = c("CD79A","MS4A1","JCHAIN","FCER2"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(t_nk_lung, features = c("CD8A","CD4","LCK","CD3D"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(t_nk_lung, features = c("CEACAM1","EGFR","EPCAM","KRAS"), reduction = "umap.harmony")

In [ ]:
options(repr.plot.width = 40, repr.plot.height = 30)
FeaturePlot(t_nk_lung, features = c('KLRB1','KLRC1','KLRD1','KLRF1','KLRK1',
                                  "NCAM1","NKG7","EOMES",
                                  'GNLY','GZMA','GZMB','GZMH','GZMK', "CD3D","NCR1"), reduction = "umap.harmony", ncol = 4)

In [ ]:
t_nk_lung  <- FindNeighbors(t_nk_lung, reduction = "harmony", graph.name = "umap.harmony.neighbors")

In [ ]:
t_nk_lung  <- FindClusters(t_nk_lung, resolution = 0.5, graph.name = "umap.harmony.neighbors")

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 9)

DimPlot(t_nk_lung, reduction = "umap.harmony", label = T, label.size = 8)

In [ ]:
t_nk_lung  <- JoinLayers(t_nk_lung)

In [ ]:
 DefaultAssay(t_nk_lung)  <- "SCT"
	
    ### Annotate the dataset with Monaco Immune dataset
counts  <- t_nk_lung@assays$SCT@layers$counts

In [ ]:
rownames(counts)  <- rownames(t_nk_lung@assays$SCT)

In [ ]:
		pred.singler <- SingleR(test = counts, ref = mid.se, assay.type.test=1,
		labels = mid.se$label.fine, fine.tune = F)
    

    	### Annotate the dataset with Wherry dataset
        pred.singler3 <- SingleR(test =  counts, ref = hpca.se,
		assay.type.test=1,
		labels = hpca.se$label.fine, fine.tune = F)

		all_labels <- data.frame(
		Monaco_single = pred.singler$labels,
		HPCA_single = pred.singler3$labels,
		
		barcode = colnames(t_nk_lung))
		
		md2 <- t_nk_lung@meta.data
		md2$barcode = colnames(t_nk_lung)

		md3 <- left_join(md2, all_labels)
rownames(md3) <- colnames(t_nk_lung)
		t_nk_lung@meta.data <- md3
		

In [ ]:
prediction_monaco  <- pred.singler$scores  %>% as.data.frame()

In [ ]:
prediction_monaco$cluster  <- unname(t_nk_lung$seurat_clusters)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 7.5)
DimPlot(t_nk_lung, raster = F, group.by = "Monaco_single", label = F, repel = T, reduction = "umap.harmony")

In [ ]:
DimPlot(t_nk_lung, raster = F, 
        cells.highlight = colnames(subset(t_nk_lung, Monaco_single == "Natural killer cells")),
        reduction = "umap.harmony")

In [ ]:
t_nk_lung$hpca_nk_cell  <- ifelse(grepl(t_nk_lung$HPCA_single, pattern = "NK_cell"), 
       t_nk_lung$HPCA_single, "Other") 

In [ ]:
t_nk_lung$hpca_nk_cell  %>% table

In [ ]:
DimPlot(t_nk_lung, group.by = "hpca_nk_cell", reduction = "umap.harmony", cols = c("red","blue","green","grey88"))

In [ ]:
DimPlot(t_nk_lung, raster = F, 
        cells.highlight = colnames(subset(t_nk_lung, hpca_nk_cell == "NK_cell")),
        reduction = "umap.harmony")

In [ ]:
DimPlot(t_nk_lung, raster = F, 
        cells.highlight = colnames(subset(t_nk_lung, hpca_nk_cell == "NK_cell:CD56hiCD62L+")),
        reduction = "umap.harmony")

In [ ]:
DimPlot(t_nk_lung, raster = F, 
        cells.highlight = colnames(subset(t_nk_lung, hpca_nk_cell == "NK_cell:IL2")),
        reduction = "umap.harmony")

In [ ]:
is_nk_cell  <- ifelse(grepl(t_nk_lung$Monaco_single, pattern = "Natural") |
                      grepl(t_nk_lung$hpca_nk_cell, pattern = "NK"), "NK_cell","Other")

In [ ]:
is_nk_cell  %>% table

In [ ]:
t_nk_lung$is_nk_cell  <- is_nk_cell

In [ ]:
DimPlot(t_nk_lung, group.by = "is_nk_cell", reduction = "umap.harmony")

In [ ]:
VlnPlot(t_nk_lung, c("CD3D","KLRK1","GZMA","CD8A","CD3E","KLRG1"), group.by = "is_nk_cell", pt.size = 0)

## T NK subset

In [ ]:
nk_lung  <- subset(t_nk_lung, is_nk_cell == "NK_cell")

In [ ]:
nk_lung <- RunUMAP(nk_lung, reduction = "harmony", dims = 1:30, reduction.name = "umap.harmony")

In [ ]:
DimPlot(nk_lung, group.by = "sample", reduction = "umap.harmony")

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
FeaturePlot(nk_lung, features = c("GZMB","GZMA","GNLY","GZMK"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(nk_lung, features = c("HAVCR2","PDCD1","KLRC1","KLRB1"), reduction = "umap.harmony")

In [ ]:

FeaturePlot(nk_lung, features = c("CST7","PRF1","IL2RB","EOMES"), reduction = "umap.harmony")

In [ ]:
nk_lung  <- FindNeighbors(nk_lung, reduction = "harmony", graph.name = "umap.harmony.neighbors")

In [ ]:
nk_lung  <- FindClusters(nk_lung, resolution = 0.5, graph.name = "umap.harmony.neighbors")

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 9)

DimPlot(nk_lung, reduction = "umap.harmony", label = T, label.size = 8)

In [ ]:
int_lung$is_nk  <- ifelse(colnames(int_lung) %in% colnames(nk_lung), "NK_cell","Other")

In [ ]:
DimPlot(int_lung, group.by = "is_nk", cols = c("red","grey88"), reduction = "umap.harmony")

In [ ]:
saveRDS(t_nk_lung, "./datasets/lung_integrated_t_nk.rds")

In [ ]:
saveRDS(int_lung, "./datasets/lung_integrated_full_filt.rds")

In [ ]:
rm(int_lung)

# Ovarian cancer

## Full object integration

In [ ]:
load("Xenium/hgsoc_1_b.rda")
load("Xenium/hgsoc_2_a.rda")


In [ ]:
int_ova  <- merge(hgsoc_1_b, hgsoc_2_a)

In [ ]:
int_ova$sample  <- substr(colnames(int_ova),12,12)

In [ ]:
int_ova$sample  <- ifelse(int_ova$sample == 1, "hgsoc_1_b","hgsoc_2_a")

In [ ]:
int_ova$sample  %>% table

In [ ]:
int_ova <- SCTransform(int_ova, assay = "Xenium")
int_ova <- RunPCA(int_ova, npcs = 30, features = rownames(int_ova))
int_ova <- RunUMAP(int_ova, dims = 1:30)
int_ova <- FindNeighbors(int_ova, reduction = "pca", dims = 1:30)
int_ova <- FindClusters(int_ova, resolution = 0.3)

In [ ]:
DimPlot(int_ova, group.by = "sample")

In [ ]:
DimPlot(int_ova)

In [ ]:
int_ova <- split(int_ova, f = int_ova$sample)

In [ ]:
int_ova <- NormalizeData(int_ova)
int_ova <- FindVariableFeatures(int_ova)
int_ova <- ScaleData(int_ova)
int_ova <- RunPCA(int_ova)

In [ ]:
int_ova <- IntegrateLayers(
  object = int_ova, method = HarmonyIntegration,
  orig.reduction = "pca", new.reduction = "harmony",
  verbose = FALSE
)

In [ ]:
int_ova <- RunUMAP(int_ova, reduction = "harmony", dims = 1:30, reduction.name = "umap.harmony")

In [ ]:
DimPlot(int_ova, group.by = "sample", reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_ova, features = c("CD79A","MS4A1","JCHAIN","FCER2"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_ova, features = c("CD8A","CD4","LCK","CD3D"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_ova, features = c("CEACAM1","EGFR","EPCAM","KRAS"), reduction = "umap.harmony")

In [ ]:
options(repr.plot.width = 40, repr.plot.height = 30)
FeaturePlot(int_ova, features = c('KLRB1','KLRC1','KLRD1','KLRF1','KLRK1',
                                  "NCAM1","NKG7","EOMES",
                                  'GNLY','GZMA','GZMB','GZMH','GZMK', "CD3D","NCR1"), reduction = "umap.harmony", ncol = 4)

In [ ]:
saveRDS(int_ova, "./datasets/ova_integrated_full_filt_harmony.rds")

In [ ]:
int_ova  <- readRDS("./datasets/ova_integrated_full_filt_harmony.rds")

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 9)

DimPlot(int_ova, reduction = "umap.harmony", label = T, label.size = 8)

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 9)

DimPlot(int_ova, reduction = "umap.harmony", label = T, label.size = 8)

In [ ]:
VlnPlot(int_ova, features =  c("nFeature_Xenium", "nCount_Xenium"), pt.size = 0)

In [ ]:
VlnPlot(int_ova, features =  c("CD3D", "LCK"), pt.size = 0)

In [ ]:
int_ova$keep  <- ifelse(int_ova$seurat_clusters %in% c(14),"Remove","Keep")

In [ ]:
DimPlot(int_ova, group.by = "keep", cols = c("grey88",'red'))

In [ ]:
ggplot(data.frame(nCount_Xenium = int_ova$nCount_Xenium,
                  nFeature_Xenium = int_ova$nFeature_Xenium,
                  seurat_clusters = int_ova$seurat_clusters,
                        exclude = ifelse(int_ova$keep == "Keep", "Keep", "Remove")), 
       aes(x = seurat_clusters, y = nFeature_Xenium)) +
  geom_violin(scale = "width", aes(fill = exclude)) + 
  ggtitle("Excluded clusters of low quality cells") + 
  theme_classic() +
  scale_fill_manual(values = c("white","red")) +
  theme(panel.background = element_blank(), 
        axis.text.x = element_text(angle = 0, hjust = 1))

In [ ]:
int_ova$keep  %>% table

In [ ]:
table(int_ova$keep, int_ova$sample)

In [ ]:
int_ova  <- subset(int_ova, keep == "Keep")

In [ ]:
int_ova <- RunUMAP(int_ova, reduction = "harmony", dims = 1:30, reduction.name = "umap.harmony")

In [ ]:
DimPlot(int_ova, reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_ova, features = c("CD79A","MS4A1","JCHAIN","FCER2"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_ova, features = c("CD8A","CD4","LCK","CD3D"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(int_ova, features = c("CEACAM1","EGFR","EPCAM","KRAS"), reduction = "umap.harmony")

In [ ]:
options(repr.plot.width = 40, repr.plot.height = 30)
FeaturePlot(int_ova, features = c('KLRB1','KLRC1','KLRD1','KLRF1','KLRK1',
                                  "NCAM1","NKG7","EOMES",
                                  'GNLY','GZMA','GZMB','GZMH','GZMK', "CD3D","NCR1"), reduction = "umap.harmony", ncol = 4)

In [ ]:
int_ova  <- FindNeighbors(int_ova)

In [ ]:
int_ova  <- FindClusters(int_ova, resolution = 0.2)

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 9)

DimPlot(int_ova, reduction = "umap.harmony", label = T, label.size = 8)

In [ ]:
FeaturePlot(int_ova, features = c("KLRF1","GNLY","CD3D","EPCAM"), reduction = "umap.harmony")

## Annotation of cell types

In [ ]:
mid.se <- celldex::MonacoImmuneData()
hpca.se  <- celldex::HumanPrimaryCellAtlasData()



In [ ]:
int_ova

In [ ]:
int_ova  <- JoinLayers(int_ova)

In [ ]:
 DefaultAssay(int_ova)  <- "SCT"
	
    ### Annotate the dataset with Monaco Immune dataset
counts  <- int_ova@assays$SCT@layers$counts

In [ ]:
rownames(counts)  <- rownames(int_ova@assays$SCT)

In [ ]:
		pred.singler <- SingleR(test = counts, ref = mid.se, assay.type.test=1,
		labels = mid.se$label.fine, fine.tune = F)
    

    	### Annotate the dataset with Wherry dataset
        pred.singler3 <- SingleR(test =  counts, ref = hpca.se,
		assay.type.test=1,
		labels = hpca.se$label.fine, fine.tune = F)

		all_labels <- data.frame(
		Monaco_single = pred.singler$labels,
		HPCA_single = pred.singler3$labels,
		
		barcode = colnames(int_ova))
		
		md2 <- int_ova@meta.data
		md2$barcode = colnames(int_ova)

		md3 <- left_join(md2, all_labels)
rownames(md3) <- colnames(int_ova)
		int_ova@meta.data <- md3
		

In [ ]:
prediction_monaco  <- pred.singler$scores  %>% as.data.frame()

In [ ]:
prediction_monaco$cluster  <- unname(int_ova$seurat_clusters)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 7.5)
DimPlot(int_ova, raster = F, group.by = "Monaco_single", label = F, repel = T, reduction = "umap.harmony")

In [ ]:
DimPlot(int_ova, raster = F, 
        cells.highlight = colnames(subset(int_ova, Monaco_single == "Natural killer cells")),
        reduction = "umap.harmony")

In [ ]:
prediction_monaco  <- prediction_monaco  %>% pivot_longer(!cluster, 
                                                          names_to = "annotation", values_to = "score")

In [ ]:
options(repr.plot.width = 17, repr.plot.height = 10)

ggplot(prediction_monaco, aes(x = cluster, y = score)) +
  geom_violin(scale = "width", aes(fill = cluster)) +
  facet_wrap(~annotation, ncol = 8) +
  ggtitle("Cluster annotations - Monaco Immune Database") + 
  theme_classic() +
  theme(panel.background = element_blank(), 
        axis.text.x = element_text(angle = 0, hjust = 1))

In [ ]:
options(repr.plot.width = 40, repr.plot.height = 10)
DimPlot(int_ova, raster = F, group.by = "HPCA_single", label = F, reduction = "umap.harmony")

## Coarse annotations

In [ ]:
int_ova  <- readRDS("./datasets/ova_integrated_full_filt_harmony.rds")

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 7)

DimPlot(int_ova, reduction = "umap.harmony", label = T, label.size = 8)

In [ ]:
int_ova$annotations_coarse  <- recode(int_ova$seurat_clusters, 
                                      "0" = "T/NK",
                                      "1" = "Tumor",
                                      "2" = "Plasma cells",
                                      "3" = "Fibroblasts",
                                      "4" = "B cells",
                                      "5" = "Myeloid cells",
                                      "6" = "Tumor"
                                      )

In [ ]:
DimPlot(int_ova, reduction = "umap.harmony", label = T, label.size = 8, group.by = "annotations_coarse")

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 4)
DotPlot(int_ova, cols = c("dodgerblue","indianred2"),
        group.by = "annotations_coarse",
        features = c(
            "CD68","FCGR2A","APOE","ITGAX",
            "CD19","BANK1","MS4A1","CD79A",
            "FN1","DCN","SPARC","SPARCL1",
            "CD27","CD38","IGHG4","IGHM",
            "CD47","EPCAM","SOS1","KRAS",
            "CD3D","CD3E","CD8A","LCK"
        )) + theme(axis.text.x = element_text(angle = 90))

## Visualization of cell types

In [ ]:
hgsoc_1_b  <- readRDS("datasets/hgsoc_1_b.rds")

In [ ]:
Idents(hgsoc_1_b)  <- hgsoc_1_b$annotations_coarse

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 16)

ImageDimPlot(hgsoc_1_b, fov = "fov", nmols = 10000, alpha = 0.5)

In [ ]:
hgsoc_2_a  <-  readRDS("datasets/hgsoc_2_a.rds")

## T NK reclustering

In [ ]:
t_nk_ova  <- subset(int_ova, seurat_clusters == 0)

In [ ]:
t_nk_ova <- RunUMAP(t_nk_ova, reduction = "harmony", dims = 1:30, reduction.name = "umap.harmony")

In [ ]:
DimPlot(t_nk_ova, group.by = "sample", reduction = "umap.harmony")

In [ ]:
FeaturePlot(t_nk_ova, features = c("CD79A","MS4A1","JCHAIN","FCER2"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(t_nk_ova, features = c("CD8A","CD4","LCK","CD3D"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(t_nk_ova, features = c("CEACAM1","EGFR","EPCAM","KRAS"), reduction = "umap.harmony")

In [ ]:
options(repr.plot.width = 40, repr.plot.height = 30)
FeaturePlot(t_nk_ova, features = c('KLRB1','KLRC1','KLRD1','KLRF1','KLRK1',
                                  "NCAM1","NKG7","EOMES",
                                  'GNLY','GZMA','GZMB','GZMH','GZMK', "CD3D","NCR1"), reduction = "umap.harmony", ncol = 4)

In [ ]:
t_nk_ova  <- FindNeighbors(t_nk_ova, reduction = "harmony", graph.name = "umap.harmony.neighbors")

In [ ]:
t_nk_ova  <- FindClusters(t_nk_ova, resolution = 0.5, graph.name = "umap.harmony.neighbors")

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 9)

DimPlot(t_nk_ova, reduction = "umap.harmony", label = T, label.size = 8)

In [ ]:
t_nk_ova  <- JoinLayers(t_nk_ova)

In [ ]:
 DefaultAssay(t_nk_ova)  <- "SCT"
	
    ### Annotate the dataset with Monaco Immune dataset
counts  <- t_nk_ova@assays$SCT@layers$counts

In [ ]:
rownames(counts)  <- rownames(t_nk_ova@assays$SCT)

In [ ]:
		pred.singler <- SingleR(test = counts, ref = mid.se, assay.type.test=1,
		labels = mid.se$label.fine, fine.tune = F)
    

    	### Annotate the dataset with Wherry dataset
        pred.singler3 <- SingleR(test =  counts, ref = hpca.se,
		assay.type.test=1,
		labels = hpca.se$label.fine, fine.tune = F)

		all_labels <- data.frame(
		Monaco_single = pred.singler$labels,
		HPCA_single = pred.singler3$labels,
		
		barcode = colnames(t_nk_ova))
		
		md2 <- t_nk_ova@meta.data
		md2$barcode = colnames(t_nk_ova)

		md3 <- left_join(md2, all_labels)
rownames(md3) <- colnames(t_nk_ova)
		t_nk_ova@meta.data <- md3
		

In [ ]:
prediction_monaco  <- pred.singler$scores  %>% as.data.frame()

In [ ]:
prediction_monaco$cluster  <- unname(t_nk_ova$seurat_clusters)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 7.5)
DimPlot(t_nk_ova, raster = F, group.by = "Monaco_single", label = F, repel = T, reduction = "umap.harmony")

In [ ]:
DimPlot(t_nk_ova, raster = F, 
        cells.highlight = colnames(subset(t_nk_ova, Monaco_single == "Natural killer cells")),
        reduction = "umap.harmony")

In [ ]:
saveRDS(t_nk_ova, "./datasets/ova_integrated_t_nk.rds")

In [ ]:
saveRDS(int_ova, "./datasets/ova_integrated_full_filt.rds")

In [ ]:
t_nk_ova$HPCA_single  %>% table

In [ ]:
t_nk_ova$hpca_nk_cell  <- ifelse(grepl(t_nk_ova$HPCA_single, pattern = "NK_cell"), 
       t_nk_ova$HPCA_single, "Other") 

In [ ]:
t_nk_ova$hpca_nk_cell  %>% table

In [ ]:
DimPlot(t_nk_ova, group.by = "hpca_nk_cell", reduction = "umap.harmony", cols = c("red","blue","green","grey88"))

In [ ]:
DimPlot(t_nk_ova, raster = F, 
        cells.highlight = colnames(subset(t_nk_ova, hpca_nk_cell == "NK_cell")),
        reduction = "umap.harmony")

In [ ]:
DimPlot(t_nk_ova, raster = F, 
        cells.highlight = colnames(subset(t_nk_ova, hpca_nk_cell == "NK_cell:CD56hiCD62L+")),
        reduction = "umap.harmony")

In [ ]:
DimPlot(t_nk_ova, raster = F, 
        cells.highlight = colnames(subset(t_nk_ova, hpca_nk_cell == "NK_cell:IL2")),
        reduction = "umap.harmony")

In [ ]:
dir.create("figures/dimplots/t_nk_ova/")

In [ ]:
for(i in rownames(t_nk_ova)){
    print(FeaturePlot(t_nk_ova, features = i, reduction = "umap.harmony"))
    ggsave(paste0("figures/dimplots/t_nk_ova/",i,".png"), width = 13, height = 10, units = "cm")
}

In [ ]:
t_nk_ova$Monaco_single  %>% table

In [ ]:
is_nk_cell  <- ifelse(grepl(t_nk_ova$Monaco_single, pattern = "Natural") |
                      grepl(t_nk_ova$hpca_nk_cell, pattern = "NK"), "NK_cell","Other")

In [ ]:
is_nk_cell  %>% table

In [ ]:
t_nk_ova$is_nk_cell  <- is_nk_cell

In [ ]:
DimPlot(t_nk_ova, group.by = "is_nk_cell", reduction = "umap.harmony")

In [ ]:
VlnPlot(t_nk_ova, c("CD3D","GNLY","GZMA","CD8A","CD8B","LCK"), group.by = "is_nk_cell", pt.size = 0)

In [ ]:
t_nk_ova  <- readRDS("./datasets/ova_integrated_t_nk.rds")

## T NK subset

In [ ]:
nk_ova  <- subset(t_nk_ova, is_nk_cell == "NK_cell")

In [ ]:
nk_ova <- RunUMAP(nk_ova, reduction = "harmony", dims = 1:30, reduction.name = "umap.harmony")

In [ ]:
DimPlot(nk_ova, group.by = "sample", reduction = "umap.harmony")

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
FeaturePlot(nk_ova, features = c("GZMB","GZMA","GNLY","GZMK"), reduction = "umap.harmony")

In [ ]:
FeaturePlot(nk_ova, features = c("HAVCR2","PDCD1","KLRC1","KLRB1"), reduction = "umap.harmony")

In [ ]:

FeaturePlot(nk_ova, features = c("CST7","PRF1","IL2RB","EOMES"), reduction = "umap.harmony")

In [ ]:
nk_ova  <- FindNeighbors(nk_ova, reduction = "harmony", graph.name = "umap.harmony.neighbors")

In [ ]:
nk_ova  <- FindClusters(nk_ova, resolution = 0.5, graph.name = "umap.harmony.neighbors")

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 9)

DimPlot(nk_ova, reduction = "umap.harmony", label = T, label.size = 8)

# Integration NK

In [ ]:
int_nk  <- merge(nk_lung, nk_ova)

In [ ]:
int_nk <- SCTransform(int_nk, assay = "Xenium")
int_nk <- RunPCA(int_nk, npcs = 30, features = rownames(int_nk))
int_nk <- RunUMAP(int_nk, dims = 1:30)
int_nk <- FindNeighbors(int_nk, reduction = "pca", dims = 1:30)
int_nk <- FindClusters(int_nk, resolution = 0.3)

In [ ]:
DimPlot(int_nk, group.by = "sample")

In [ ]:
DimPlot(int_nk)

In [ ]:
int_nk <- split(int_nk, f = int_nk$sample)

In [ ]:
int_nk <- NormalizeData(int_nk)
int_nk <- FindVariableFeatures(int_nk)
int_nk <- ScaleData(int_nk)
int_nk <- RunPCA(int_nk)

In [ ]:
int_nk <- IntegrateLayers(
  object = int_nk, method = HarmonyIntegration,
  orig.reduction = "pca", new.reduction = "harmony",
  verbose = FALSE
)

In [ ]:
int_nk <- RunUMAP(int_nk, reduction = "harmony", dims = 1:30, reduction.name = "umap.harmony")

In [ ]:
DimPlot(int_nk, group.by = "sample", reduction = "umap.harmony")

In [ ]:
VlnPlot(int_nk, group.by = "sample", features = c("CST7","GZMK","GZMA","NCAM1","PDCD1","HAVCR2","TOX"))

In [ ]:
DotPlot(int_nk, group.by = "sample", features = c("CST7","GZMK","GZMA","NCAM1","PDCD1","HAVCR2","TOX"))

In [ ]:
avgexp  <- AverageExpression(int_nk, return.seurat = F, group.by = "sample", assays = "SCT")

In [ ]:
avgexp3  <- as.data.frame(avgexp$SCT)  %>% rownames_to_column("gene")  %>% pivot_longer(!gene, names_to = "sample",
                                                                                       values_to = "value")  %>% 
mutate(cancer = substr(sample,1,5))

In [ ]:
avgexp3

In [ ]:
ggtheme <- function() {
  theme(
    axis.text = element_text(size = 20),
    axis.title = element_text(size = 20),
    text = element_text(size = 20, colour = "black"),
    legend.text = element_text(size = 20),
    legend.key.size =  unit(10, units = "points")
    
  )
}


In [ ]:
options(repr.plot.width = 3, repr.plot.height = 3.7)

for(i in levels(factor(avgexp3$gene))){
   p  <- avgexp3  %>% dplyr::filter(gene == i)  %>% 
    ggplot(aes(x = cancer, y = value)) +
     geom_dotplot(aes(fill = cancer), binaxis='y', stackdir='center', dotsize = 2) +
    ggtitle(i) + 
     theme_classic() + NoLegend() + ggtheme() + xlab("") + ylab("Average Expression")
    
    
    suppressWarnings(print(p))
}

In [ ]:
library(pheatmap)

In [ ]:
avgexp2  <- t(avgexp$SCT[which(rownames(avgexp$SCT) %in% c())])

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 3.7)
pheatmap(t(avgexp$SCT), main = "", 
         scale = "column", cluster_cols = T, cluster_rows = T,
        color=colorRampPalette(c("dodgerblue", "grey95", "indianred2"))(50), 
         border_color = "white",
         width = 8, height = 3,
                  fontsize = 9)

### NK cell subset from Fig 1I

In [ ]:
load("./datasets/nk.subset.rda")

In [ ]:
nk.subset$Group  %>% table

In [ ]:
nk.subset$Group1  %>% table

In [ ]:
nk.subset$NK.purity  %>% table

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(nk.subset, group.by = "NK.purity")

In [ ]:
names(nk.subset@reductions)

In [ ]:
for(i in names(nk.subset@reductions)[3:8]){
    print(DimPlot(nk.subset, group.by = "NK.purity", reduction = i))
    }

In [ ]:
DefaultAssay(nk.subset)

In [ ]:
Idents(nk.subset)  <- nk.subset$NK.purity
mrk_nk  <- FindAllMarkers(nk.subset)

In [ ]:
mrk_nk  %>% dplyr::filter(gene %in% c(rownames(nk_lung)) & avg_log2FC > 0)

In [ ]:
nk.subset$Group  %>% table

In [ ]:
avgexp  <- AverageExpression(nk.subset, return.seurat = F, group.by = "Group", assays = "RNA")

In [ ]:
avgexp3  <- as.data.frame(avgexp$RNA)  %>% rownames_to_column("gene")  %>% pivot_longer(!gene, names_to = "sample",
                                                                                       values_to = "value")  %>% 
mutate(cancer = substr(sample,1,2))  %>% dplyr::filter(gene %in% c(rownames(nk_lung)))

In [ ]:
avgexp3

In [ ]:
options(repr.plot.width = 3, repr.plot.height = 3.7)

for(i in c("SELL","NCAM1","IL10","CTLA4","FOXP3","HAVCR2","IDO1")){
   p  <- avgexp3  %>% dplyr::filter(gene == i)  %>% 
    ggplot(aes(x = cancer, y = value)) +
     geom_dotplot(aes(fill = cancer), binaxis='y', stackdir='center', dotsize = 2) +
    ggtitle(i) + 
     theme_classic() + NoLegend() + ggtheme() + xlab("") + ylab("Average Expression")
    
    
    suppressWarnings(print(p))
}

## NK subtype score

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 3.7)
FeaturePlot(nk.subset, features = "GZMK", reduction = "umap.rpca")
FeaturePlot(nk.subset, features = "NCAM1", reduction = "umap.rpca")
FeaturePlot(nk.subset, features = "CST7", reduction = "umap.rpca")
FeaturePlot(nk.subset, features = "TBX21", reduction = "umap.rpca")


In [ ]:
score_inhib  <- mrk_nk  %>% dplyr::filter(gene %in% c(rownames(nk_lung)) & avg_log2FC > 0 & cluster == "Pure.cl8" & p_val_adj < 0.05)  %>% pull(gene)

In [ ]:
score_inhib

In [ ]:
score_cytotox  <- mrk_nk  %>% 
dplyr::filter(gene %in% c(rownames(nk_lung)) & avg_log2FC > 0 & 
              cluster == "Pure.cl12" & p_val_adj < 0.05)  %>% pull(gene)

In [ ]:
score_cytotox

In [ ]:
int_nk@assays$SCT$scale.data.hgsoc_2_a  <- NULL

In [ ]:
int_nk  <- JoinLayers(int_nk)

In [ ]:
int_nk  <- AddModuleScore(
  object = int_nk,
  features = list(c(score_inhib)),
  assay = "SCT",
    nbin = 10,
    ctrl = 10,
    replace = TRUE,
  name = 'NK_inhib_')

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 7)
VlnPlot(int_nk, features = "NK_inhib_1", pt.size = 0, group.by = "sample") + 
  stat_summary(fun = "mean",
               geom = "crossbar", 
               width = 0.5,
               colour = "black")  + NoLegend() + ggtheme() + xlab("")

In [ ]:
int_nk  <- AddModuleScore(
  object = int_nk,
  features = list(c(score_cytotox)),
  assay = "SCT",
    nbin = 10,
    ctrl = 10,
    replace = TRUE,
  name = 'NK_cytotox_')

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 7)

VlnPlot(int_nk, features = "NK_cytotox_1", pt.size = 0, group.by = "sample") + 
  stat_summary(fun = "mean",
               geom = "crossbar", 
               width = 0.5,
               colour = "black") + NoLegend() + ggtheme() + xlab("")

# Tertiary lymphoid structures

In [ ]:
tls_chemokine_signature  <- c("CCL2", "CCL3", "CCL4", "CCL5", "CCL8", "CCL19", "CCL21", "CXCL9", "CXCL10", "CXCL11", "CXCL13")

In [ ]:
ls()

In [ ]:
load("Xenium/hgsoc_1_b.rda")
load("Xenium/hgsoc_2_a.rda")

In [ ]:
int_nk$barcode  <- substr(colnames(int_nk),1,nchar(colnames(int_nk))-2)

In [ ]:
table(colnames(hgsoc_1_b)  %in% int_nk$barcode)

# Visualization of NK cells

## hgsoc_1_b

In [ ]:
options(future.globals.maxSize = 10000 * 1024^2)

In [ ]:
nk_hgsoc_1_b  <- subset(int_nk, sample == "hgsoc_1_b")

In [ ]:
colnames(nk_hgsoc_1_b)  <- nk_hgsoc_1_b$barcode

In [ ]:
colnames(nk_hgsoc_1_b) %in% colnames(hgsoc_1_b)  %>% table

### NK cells in tissue

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 16)
highlightCells(highlight_obj = nk_hgsoc_1_b, within_obj = hgsoc_1_b, 
               size = 2, save_plot = FALSE, color_palette = c("darkblue","yellow"))

## hgsoc_2_a

In [ ]:
nk_hgsoc_2_a  <- subset(int_nk, sample == "hgsoc_2_a")

In [ ]:
colnames(nk_hgsoc_2_a)  <- nk_hgsoc_2_a$barcode

In [ ]:
colnames(nk_hgsoc_2_a) %in% colnames(hgsoc_2_a)  %>% table

### NK cells in tissue

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 16)
highlightCells(highlight_obj = nk_hgsoc_2_a, within_obj = hgsoc_2_a, 
               size = 2, save_plot = FALSE, color_palette = c("darkblue","yellow"))

### Zoom to TLS

In [ ]:
cropped.coords <- Crop(hgsoc_2_a[["fov"]], x = c(1800, 2000), y = c(2200, 2400), coords = "plot")
hgsoc_2_a[["zoom"]] <- cropped.coords
# visualize cropped area with cell segmentations & selected molecules
DefaultBoundary(hgsoc_2_a[["zoom"]]) <- "segmentation"
ImageDimPlot(hgsoc_2_a, fov = "zoom", axes = TRUE, border.color = "white", size = 3,  border.size = 0.1, cols = "polychrome",
    coord.fixed = FALSE, molecules = c("GNLY", "CD3E", "LCK", "CST7", "FOXP3"), nmols = 20000)

In [ ]:
ImageDimPlot(hgsoc_2_a, fov = "zoom", axes = TRUE, border.color = "white", size = 20,  border.size = 0.1, cols = "polychrome",
    coord.fixed = FALSE, molecules = c("GNLY", "CD3E", "LCK", "CST7", "FOXP3"), nmols = 20000)

In [ ]:
ImageFeaturePlot(hgsoc_2_a, features = "TBX21", fov = "fov", axes = TRUE,
                 border.color = "white",  border.size = 0.1, 
    coord.fixed = FALSE)

In [ ]:
ImageFeaturePlot(hgsoc_2_a, features = "GNLY", fov = "zoom", axes = TRUE,
                 border.color = "white", size = 20,  border.size = 0.1, 
    coord.fixed = FALSE)

In [ ]:
cells = colnames(nk_hgsoc_2_a), 

In [ ]:
ImageDimPlot(hgsoc_2_a, cells = colnames(nk_hgsoc_2_a), fov = "zoom", axes = TRUE, border.color = "white", size = 20,  border.size = 0.1, cols = "polychrome",
    coord.fixed = FALSE, molecules = c("GNLY", "CD3E", "LCK", "CST7", "FOXP3"), nmols = 20000)

In [ ]:
int_nk$samplemplemple  %>% table

## nsclc_1B

In [ ]:
nk_nsclc_1B  <- subset(int_nk, sample == "nsclc_1B")

In [ ]:
colnames(nk_nsclc_1B)  <- nk_nsclc_1B$barcode

In [ ]:
colnames(nk_nsclc_1B)  %>% head

In [ ]:
colnames(NSCLC.1B)  %>% head

In [ ]:
colnames(nk_nsclc_1B) %in% colnames(NSCLC.1B)  %>% table

In [ ]:
ls()

### NK cells in tissue

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 16)
highlightCells(highlight_obj = nk_nsclc_1B, within_obj = NSCLC.1B, 
               size = 2, save_plot = FALSE, color_palette = c("darkblue","yellow"))

## nsclc_2_a

In [ ]:
ls()

In [ ]:
nk_nsclc_2_a  <- subset(int_nk, sample == "nsclc_2_a")

In [ ]:
colnames(nk_nsclc_2_a)  <- nk_nsclc_2_a$barcode

In [ ]:
colnames(nk_nsclc_2_a)  %>% head

In [ ]:
colnames(nsclc_2_a)  %>% head

In [ ]:
load("Xenium/nsclc_2_a.rda")


In [ ]:
colnames(nsclc_2_a)  %>% head

In [ ]:
colnames(nk_nsclc_2_a) %in% colnames(nsclc_2_a)  %>% table

### NK cells in tissue

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 16)
highlightCells(highlight_obj = nk_nsclc_2_a, within_obj = nsclc_2_a, 
               size = 3, save_plot = FALSE, color_palette = c("darkblue","yellow"))

In [ ]:
ls()

In [ ]:
rm(hgsoc_2_a)